# So you want to automate translation? Let's do it using Python! 

## Purpose 
This notebook exists to automate translation of languages found in data sets that would otherwise require manual copy and pasting into a translation engine (such as Google Translate). 

We want to automate the table below. Doing this manually isn't daunting when it is only four rows, but it become burdensome when it becomes hundreds or thousands or rows. 
<table>
  <tr>
    <th>Original Text</th>
    <th>Translated Text</th>
  </tr>
  <tr>
    <td>Bonjour</td>
    <td>Hello</td>
  </tr>
  <tr>
    <td>Au revoir</td>
    <td>Goodbye</td>
  </tr>
  <tr>
    <td>Merci</td>
    <td>Thank you</td>
  </tr>
</table>



## Outline of this Jupyter Notebook 
1. Install and import packages
1. Connect to data source 
1. Selection of columns to translate using a list 
1. Select input and output language to use (if not specified, will use auto-detection and translate to English)
1. Run translation and wait! 
1. Export new dataset 



## Any location that has the following <span style="color: red;">ACTION REQUIRED</span>, immediately afterwards you need to edit or make a change for the script to run. 

## Any location that has the following <span style="color: blue;">ACTION OPTIONAL</span>, immediately afterwards you can make an optional edit or make a change to customize the script. 


## 1. Install and Import Packages

In [ ]:
# This block of code creates a function (most everything that follows "def" is that function) 
# This function checks to see if the primary translation package, googletrans, is installed, if it isn't, it will be installed. 

# Using Anaconda ensures many standard packages are already installed (such as importlib and pandas among many others)
#Becuase of the comment immediately above, this is why you do not have to run this function for most of the pakcages you use 
import importlib

def install_package(package_name):
    try:
        importlib.import_module(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        print(f"{package_name} is not installed. Installing now...")
        try:
            import pip
            pip.main(['install', package_name])  # For pip versions < 10.0.0
        except AttributeError:
            import subprocess
            subprocess.check_call(['pip', 'install', package_name])  # For pip versions >= 10.0.0
        print(f"{package_name} has been installed.")

#This runs the "install_package" function to see if "googletrans" package is installed. 
install_package("googletrans")

In [ ]:
import pandas as pd 
from googletrans import Translator, constants, LANGUAGES
translator = Translator()

## 2. Connect to Data Source

## <span style="color: red;">ACTION REQUIRED</span>

In [ ]:
#df is the dataframe, or the data we are using 
#you must change the file path so it goes to the data you want to translate
df=pd.read_excel(r"C:\Users\path\to\the\data.xlsx")

## 3. Columns to Translate 
- If you need to translate all the columns: 
    - have the code block immediately below be `cols_to_translate=list(df)`

## <span style="color: red;">ACTION REQUIRED</span>

In [ ]:
#After adding columns you would like, you can run this cell
cols_to_translate=list(df)

### Prepare/Clean the Data for Translation

In [ ]:
#This takes all of the columns listed above in cols_to_translate and makes sure to keep them in their orignal state 
#We do this to retain the oringal text, hence 'ORIG_TXT' in the dataset
prefix = 'ORIG_TXT'

for col in cols_to_translate:
    new_col_name = prefix + col
    df[new_col_name] = df[col].copy()

    
# This now goes through and gets the unique entries in those columns to reduce the burden of translation

#first create a dataframe that takes only the columns we are concerned with
selected_cols2 = df.filter(cols_to_translate, axis=1)
# create a set to store unique entries
unique_entries = set(selected_cols2.values.ravel())
unique_entries = {x for x in unique_entries if pd.notna(x)}
# create a dictionary to store translations
translations = {}

## 4. Select Languages for translation 

The next code block's output, shows all of the languages available for translation. If you leave `source_lang` blank, it will try to use auto-detection to determine the language. 

When selecting which languages to use, ensure to use the two letter code. For example, if you want to translate from Croation: <br>
`source_lang="co"` <span style="color: darkgreen;">will work</span> <br>
~~`source_lang="Croation"`~~ <span style="color: darkred;">will not work</span> <br>

If you want to auto-detect the language do the following: <br>

`source_lang=""` <br>


To change the target language (i.e. the language that will be outputted) change the two letter code next to `target_lang`


In [ ]:
# Print all supported languages
for code, language in LANGUAGES.items():
    print(f"{code}: {language}")

##  <span style="color: blue;">ACTION OPTIONAL</span>

In [ ]:
source_lang=""
target_lang='en'

## 5. Run Translation and wait

The next code block does the actual translating. It may take some time to translate. Even minutes. Just give it time. 

In [ ]:
#DO NOT EDIT CODE IN THIS BLOCK UNLESS YOU ARE CERTAIN THERE IS AN ERROR YOU CAN CORRECT
# loop over the unique entries and translate them
for entry in unique_entries:
    try:
             
        # check if entry is already in translations dictionary
        if entry in translations:
            continue

        # translate the entry
        if source_lang=="":
            translated_entry = translator.translate(entry, dest=target_lang).text
        else:
            translated_entry = translator.translate(entry, src=source_lang, dest=target_lang).text
        
        # add the entry and its translation to the dictionary
        translations[entry] = translated_entry
    except Exception as e:
        if "ReadTimeout" in str(e):
            continue  # Continue to the next iteration of the loop

In [ ]:
# prints the translations dictionary
print(translations)

In [ ]:
# replaces the original entries with their translations in the dataframe
selected_cols2.replace(translations, inplace=True)
#Neat method to take columns that have the same name from selected_cols2 and put them into the orignal dataframe with the translated comments
df.update(selected_cols2)

In [ ]:
#displays only the translated columns
df[cols_to_translate]

## 6. Export Translated Data
Unless otherwise specified in the `df.to_excel(....`, the data will be exported to the folder from the output of the code cell `pwd`

In [ ]:
#this stands for print working directory 
pwd

 ## <span style="color: blue;">ACTION OPTIONAL</span>

In [ ]:
# Feel free to change the thing below to a name you prefer
name='Data'

#export to Excel
df.to_excel(name+'_translated.xlsx')